In [5]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import openpyxl
from time import time, localtime, strftime

class ExplorerScreenShot:
    def __init__(self, workbook, sheet_name):
        self.workbook = openpyxl.load_workbook(filename=workbook, data_only=True)
        self.sheet_name = sheet_name
        self.sheet = self.workbook[sheet_name]

        self.options = webdriver.ChromeOptions()
        self.options.add_argument("headless")
        self.options.add_argument("--window-size=1920,1440")
        self.options.add_argument(f'user-agent=Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.50 Safari/537.36')
        
        self.driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=self.options)
        
        self.network_map = {
            'ETH': "https://etherscan.io/tx/",
            'ERC20': "https://etherscan.io/tx/",
            "BTC": "https://www.blockchain.com/btc/tx/",
        }


    def logger(func):
        def wrapper(*args, **kwargs):
            start = time()
            start_time = strftime("%Y-%m-%d %I:%M:%S", localtime(start))
            print(f"starts now! {start_time}")
            func(*args, **kwargs)
            end = time()
            end_time = strftime("%Y-%m-%d %I:%M:%S", localtime(end))
            print(f"completed at {end_time}, took {round(end - start, 2)} seconds")
        return wrapper
    
    @logger
    def get_screenshot(self, target_network, start = 1):
        for i in range(start, self.sheet.max_row + 1):
            network = self.sheet[f"K{str(i)}"].value
            if network in target_network:
                tx_hash = self.sheet[f"L{str(i)}"].value
                print(f"network: {network}, tx_hash: {tx_hash}")
                self.driver.implicitly_wait(6)
                self.driver.get( self.network_map[network] + tx_hash )
                self.driver.implicitly_wait(6)
                self.driver.save_screenshot(f"screenshots/{self.sheet_name}_{network}_row{i}.png")
                self.driver.implicitly_wait(6)

        self.driver.quit()

In [2]:
filename ="screenshots/OKEx Funding Account History_v5.xlsx"
es = ExplorerScreenShot(filename, "OKEx Funding 출금 History")
es.get_screenshot(["ERC20", "BTC"], 1)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 103.0.5060
[WDM] - Get LATEST chromedriver version for 103.0.5060 google-chrome
[WDM] - About to download new driver from https://chromedriver.storage.googleapis.com/103.0.5060.53/chromedriver_mac64.zip
[WDM] - Driver has been saved in cache [/Users/bbrick/.wdm/drivers/chromedriver/mac64/103.0.5060.53]


starts now! 2022-06-28 06:54:05
network: ERC20, tx_hash: 0x275512c3aa3e8fb4d3b9920d897bfd18b40cde2d57dbec8007c3fc7d41224edb
network: ERC20, tx_hash: 0x79b2741320297b8d3432faeed93ba670464284e171d9af95db1918162f68c3c4
network: ERC20, tx_hash: 0xd82136432629a7cab8dfd3647efd2fae09a801a7fb7138d767231700c08cf095


KeyboardInterrupt: 

In [7]:
from openpyxl.utils import get_column_letter
# 엑셀에 이미지 붙이기
filename = "terra_anchor입금_스샷.xlsx"
workbook = openpyxl.load_workbook(filename=filename, data_only=True)
ws = workbook.create_sheet("Peertec TMP")

# 열 맞추기
ws.column_dimensions['B'].width = 195.7


In [8]:
from openpyxl.styles import Font, PatternFill
from openpyxl.drawing.image import Image
import json
import datetime

with open("anchor_deposit/Peertec Terra Anchor deposit.json", "r") as f:
    json_data = json.load(f)
# 이미지 높이 10.1 인치, 가로 13.46인치
# L까지
tmp_datetime = datetime.datetime(2022, 4, 8, 11, 14, 56)

gray_color = PatternFill('solid', 'c8c8c8')
green_color = PatternFill('solid', '80E12A')
orange_color = PatternFill('solid', 'ffaa28')
blue_color = PatternFill('solid', '9bc3ff')

# 행 너비 어떻게 가져올 수 있을까?

bold =  Font(color="000000", bold=True, size = 12)
white_color_bold = Font(color="FFFFFF", bold=True, size = 14)

def paint(range, color):
    for cell in ws[range][0]:
        cell.fill = color

paint("A2:p2", gray_color)
ws["A2"].value = tmp_datetime.strftime('%Y-%m-%d')
ws["A2"].font = white_color_bold

paint("A3:p3", green_color)
ws["A3"].value = "Terra Anchor Deposit"
ws["A3"].font = bold

image = Image('screenshots/peertec_anchor_deposit/Anchor deposit_2022-06-05 04:44:27.png')
image.height = 738
image.width = 1295.7
ws.add_image(image, "a4")

# A얄 19.57, 나머지 10

for cell in ws["a44:p44"][0]:
    cell.fill = orange_color
ws["A44"].value = "Description"
ws["A44"].font = bold

for cell in ws["a45:p45"][0]:
    cell.fill = blue_color
ws["A45"].value = """
2022.04.16 19:07:22, 12:13:53

Peertec Terra 지갑에서
Anchor Market으로 45,685, 598,260 USTC 입금
"""
ws["A45"].font = bold












In [21]:
workbook.save(f"테라스샷 tmp.xlsx")